# Prerequisites

## For Server
- git clone https://github.com/most-medicalrecord -b develop
- cd most-medicalrecord
- make develop
- make sync
- make run

##For testing API with this notebook## 
- **Requests** library <http://docs.python-requests.org/en/latest/>

## For testing pyEHR API endpoint ##
- Download ova VirtualMachine from <http://156.148.18.70/files/pyEHR.ova>
- Run VirtualMachine (virtualbox or vmware importing vmdk virtual disk in new virtual machine)
- Configure VirtualMachine networking with NAT or BRIDGE
- Wait for services start (up to 5 minutes)
- Test dbservice status calling <http://virtualmachine:8080/check/status/dbservice>
- Configure PYEHR_DB_SERVICE_IP on django settings.py
- stop SERVER and restart with **make run**

In [5]:
import requests
import json

server_port = 8000

In [6]:
# Get Oauth2 Access Token specifying existing taskgroup_uuid

arguments = {
    'client_id': '8c96bf8cea26fa555fa8',
    'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
    'grant_type': 'password',
    'username': 'admin',
    'password': 'admin',
    'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
}
r = requests.post(
    'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
    data=arguments
)
print r.text
access_token = json.loads(r.text)['access_token']

{"access_token": "97ee06852b64af3e226415a4f7e76d78a914741e", "token_type": "Bearer", "expires_in": 31535999, "refresh_token": "0723836934b2de0ddd9f81942b8c42bd6fc31bbd", "scope": "read"}


In [9]:
#Test authenticated url

payload = {'access_token': access_token}
r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/ping/auth/'.format(server_port=server_port), 
    params=payload
)
print r.text

authenticated pong


In [25]:
#Create new patient with existing demographics and ehr uuid

payload = {'access_token': access_token, 'demographic_uuid': 'afbab3d293b643833d35cb972f26642cf77f1db4'}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text

{"patient": {"demographic_uuid": "afbab3d293b643833d35cb972f26642cf77f1db4", "uuid": "blonlue77fhsf5dgmykvs2cio3h5yi6f"}, "success": true}


In [39]:
#Retrieve all patients for taskgroup
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
if len(data['patients']) > 0:
    patient_uuid = data['patients'][0]['uuid']

{"patients": [{"demographic_uuid": "afbab3d293b643833d35cb972f26642cf77f1db4", "uuid": "blonlue77fhsf5dgmykvs2cio3h5yi6f"}], "success": true}


In [38]:
#Retrieve patient by uuid
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload
)
print r.text

{"errordata": {"message": "request patient does not exists", "code": 501}, "success": false}


In [33]:
#Update patient data by uuid
payload = {'access_token': access_token, 'demographic_uuid': 'a'}

r = requests.put(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload, data=data
)
print r.text

{"patient": {"demographic_uuid": "a", "uuid": "drpp32jnmapx3roaf5j7k46saj2ynuba"}, "success": true}


In [40]:
#Delete patient by uuid
payload = {'access_token': access_token}

r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.
    format(server_port=server_port, patient_uuid="blonlue77fhsf5dgmykvs2cio3h5yi6f"), 
    params=payload, data=data
)
print r.text

{"patient": {"demographic_uuid": "afbab3d293b643833d35cb972f26642cf77f1db4", "uuid": "blonlue77fhsf5dgmykvs2cio3h5yi6f"}, "success": true}
